In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import random

In [ ]:
df = pd.read_csv('/content/poems.csv', header=None, names=['text'])
text = " ".join(df['text'].tolist())

print(text[:500])

Through the forest deep, where shadows linger long, the night sings its song. In the stillness of night, the moon whispers its light. Under the velvet sky, dreams take flight on wings of stars. A single leaf falls, carried away by the gentle breeze. In the stillness of night, the moon whispers its light. Love is the light that guides us through the darkest times. A single leaf falls, carried away by the gentle breeze. A whisper of hope in the silence of the dawn. A whisper of hope in the silence


In [ ]:
vocab = sorted(set(text))
char_to_idx = {c: i for i, c in enumerate(vocab)}
idx_to_char = {i: c for i, c in enumerate(vocab)}

In [ ]:
max_len = 100  # length of each input sequence
step = 5       # step size for moving window

sentences = []
next_chars = []

for i in range(0, len(text) - max_len, step):
    sentences.append(text[i: i + max_len])
    next_chars.append(text[i + max_len])

X = np.zeros((len(sentences), max_len, len(vocab)), dtype=bool)
y = np.zeros((len(sentences), len(vocab)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_idx[char]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1


In [ ]:
model = Sequential()
model.add(GRU(128, input_shape=(max_len, len(vocab))))
model.add(Dense(len(vocab)))
model.add(Activation('softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_3 (GRU)                     │ (None, 128)            │        63,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 35)             │         4,515 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 35)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 67,875 (265.14 KB)

 Trainable params: 67,875 (265.14 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X, y, batch_size=128, epochs=30)

Epoch 1/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - loss: 0.6678
Epoch 2/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0550
Epoch 3/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0509
Epoch 4/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.0754
Epoch 5/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.4613
Epoch 6/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.3188
Epoch 7/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0918
Epoch 8/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 1.7416
Epoch 9/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 1.8898
Epoch 10/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 2.0317
Epoch 11/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 2.0397
Epoch 12/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 2.1750
Epoch 13/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 2.0550
Epoch 14/30
1389/1389 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 2.0739
Epoch 15/30
138

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature  # add epsilon to avoid log(0)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length=400, temperature=0.5):
    start_idx = random.randint(0, len(text) - max_len - 1)
    generated = ''
    sentence = text[start_idx: start_idx + max_len]
    generated += sentence

    for _ in range(length):
        x_pred = np.zeros((1, max_len, len(vocab)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_idx[char]] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_idx = sample(preds, temperature)
        next_char = idx_to_char[next_idx]

        generated += next_char
        sentence = sentence[1:] + next_char

    return generated

In [ ]:
print(generate_text(length=500, temperature=0.3))

, dreams take flight on wings of stars. Love is the light that guides us through the darkest times. In the stillness of the garden, flowers bloom with grace. In the stillness of the garden, flowers bloom with grace. The mountain peaks kiss the sky, standing proud and wise. A whisper of hope in the silence of the dawn. In the stillness of the garden, flowers bloom with grace. The mountain peaks kiss the sky, standing proud and wise. A whisper of hope in the silence of the dawn. In the stillness of the garden, flowers bloom with grace. The mountain peaks kiss the sky, standing proud and wise. Th
